# Combine analysis results

In [ ]:
import os
import pandas as pd

figure_dir = os.path.join('..', '..', 'figures')
os.makedirs(figure_dir, exist_ok=True)

evaluation_dir = os.path.join('..', '..', 'evaluation')
fine_tuned_path = os.path.join(evaluation_dir, 'efficeint_phys_fine_tuned', 'analysis.csv')
pretrained_path = os.path.join(evaluation_dir, 'pretrained', 'analysis.csv')
unsupervised_path = os.path.join(evaluation_dir, 'unsupervised', 'analysis.csv')

fine_tuned = pd.read_csv(fine_tuned_path)
pretrained = pd.read_csv(pretrained_path)
unsupervised = pd.read_csv(unsupervised_path)

## Harmonize analysis results

In [ ]:
# Only keep roi==chest, because it is the most accurate
unsupervised = unsupervised[unsupervised['roi'] == 'chest']

# Remove roi column
unsupervised = unsupervised.drop(columns=['roi'])

# Rename method to model
unsupervised = unsupervised.rename(columns={'method': 'model'})

In [ ]:
# Rename model_id to model
fine_tuned = fine_tuned.rename(columns={'model_id': 'model'})

In [ ]:
# Merge all dataframes
analysis = pd.concat([fine_tuned, pretrained, unsupervised])

metrics = [
    'pk_error',
    'cp_error',
    'nfcp_error',
    'fft_error',
    'distance_mse',
    'distance_pearson',
    'distance_dtw',
]

# Only keep relevant metrics
analysis = analysis[analysis['metric'].isin(metrics)]

## Compare all models

In [ ]:
analysis

In [ ]:
model_scores = []

for model in analysis['model'].unique():
    model_data = analysis[analysis['model'] == model]

    for metric in metrics:
        metric_data = model_data[model_data['metric'] == metric]
        if len(metric_data) != 0:
            model_scores.append({
                'model': model,
                'metric': metric,
                'mean': metric_data['result'].mean(),
                'std': metric_data['result'].std(),
            })

model_scores = pd.DataFrame(model_scores)
model_scores

In [ ]:
# Determine the best model for each metric
best_models = []

for metric in metrics:
    metric_data = model_scores[model_scores['metric'] == metric]

    if metric == "distance_pearson":
        value = metric_data['mean'].max()
    else:
        value = metric_data['mean'].min()

    best_model = metric_data[metric_data['mean'] == value]
    best_models.append(best_model)

best_models = pd.concat(best_models)
best_models

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(len(metrics), 1, figsize=(20, 20))

# Add some space between the plots
fig.tight_layout(pad=10.0)

# Add some space between the plots
fig.tight_layout(pad=5.0)

fig.suptitle('Comparison of models')

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    scores = model_scores[model_scores['metric'] == metric]

    for _, data in scores.iterrows():
        ax.bar(data['model'], data['mean'], yerr=data['std'], capsize=5)

    ax.set_title(metric)
    ax.set_ylabel(metric)
    ax.set_xlabel('Model')

    # Rotate the x-axis labels
    for tick in ax.get_xticklabels():
        tick.set_rotation(0)

# Store the plot as svg
plot_file = os.path.join(figure_dir, 'model_comparison.svg')
plt.savefig(plot_file, format='svg')